<a href="https://colab.research.google.com/github/objectin/Goorm-AI-Team-Project/blob/main/mrc_bigbird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets tokenizers wandb evaluate

In [2]:
import datasets
from datasets import load_dataset

from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

from transformers import pipeline
import json
from tqdm import tqdm, trange
from collections import defaultdict

In [3]:
SEED = 42
MODEL_NAME = "monologg/kobigbird-bert-base"

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB = True
except:
    COLAB = False

In [5]:
!unzip -o "/content/drive/MyDrive/Colab Notebooks/custom_dataset.zip"

'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [6]:
dataset_plus = load_dataset('ko_nia_normal_squad_all', split='train')
dataset_plus_fix = dataset_plus.filter(
    lambda example: example['question'].endswith(('는?', '가?', '은?', '나?', '요?'))
)

squad_plus = dataset_plus_fix.train_test_split(0.91, seed=SEED)
# filter: ('래?', '야?', '어?', '나?', '러?', '해?', '대?', '지')

squad_plus

Found cached dataset ko_nia_normal_squad_all (C:/Users/sms20/.cache/huggingface/datasets/ko_nia_normal_squad_all/ko_nia_normal_squad_all/0.0.1/f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb)
Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-bbef8f5762ec5d29.arrow
Loading cached split indices for dataset at C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-31de2438c36f9fac.arrow and C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-e90f09d99b3ad2ad.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 12211
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 123477
    })
})

In [7]:
dataset = load_dataset('custom_squad_v2', split='train')
squad = dataset.train_test_split(0.1, seed=SEED)

squad_augmented = datasets.concatenate_datasets([squad['train'], squad_plus['train']])
# squad['validation'] = squad['test']
# del squad['test']

Found cached dataset custom_squad_v2 (C:/Users/sms20/.cache/huggingface/datasets/custom_squad_v2/custom-squad_v2/0.0.1/429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e)
Loading cached split indices for dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-93567b845d8c56f7.arrow and C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-20d5ebf42d407cce.arrow


In [8]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10833
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1204
    })
})

In [9]:
squad_augmented

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 23044
})

In [10]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  SK텔레콤(대표이사 사장 박정호)은 20일부터 갤럭시S20 예약 판매에 돌입하고 오는 27일부터 개통을 시작한다고 밝혔다. 갤럭시S20 플러스 아우라 블루 컬러는 전 세계에서 오직 SK텔레콤에서만 구매 가능하다. 블루 컬러는 기존에 갤럭시 단말을 구매한 고객들로부터 큰 인기를 끌었다. SK텔레콤은 갤럭시S20 예약 판매를 맞아 온∙오프라인 채널에서 대대적인 ‘블루 마케팅’을 펼친다. 먼저 SK텔레콤 공식 온라인샵 T월드 다이렉트에서 갤럭시S20을 예약 및 구매한 고객 가운데 선착순 3천명에게 ‘아우라 블루 패키지’를 제공한다. ‘아우라 블루 패키지’는 갤럭시S20 단말과 아우라 블루 에코백, 신세계 상품권 3만원으로 구성돼 있다. T월드 다이렉트에서 예약한 모든 고객에게는 ▲PANTONE 액세서리 5종 ▲블루투스 이어폰 ▲맨프로토 미니 삼각대 ▲고속충전 지원 액세서리 총 8종의 사은품 가운데 1종을 증정한다. 또 예약 후 개통한 고객 가운데 100명을 추첨해 삼성 에어드레서, 무풍큐브, 블루스카이 공기청정기 등을 제공한다. 매장 방문이 어려운 고객은 T월드 다이렉트에서 ‘오늘도착’ 서비스를 신청하면, 원하는 시간과 장소에서 갤럭시S20을 손쉽게 받아 볼 수 있다. 전문 상담사가 직접 고객을 찾아가 개통부터 데이터 이전까지 마무리해 준다. 갤럭시S20을 저렴하게 구매하고 싶은 고객은 ▲제휴카드 더블할인 ▲T안심보상 ▲T모아쿠폰 등 다양한 구매 혜택을 활용해 최대 130만원까지 할인 받을 수 있다. 추후에도 새 갤럭시 단말로 기기변경을 원하는 고객은 ‘5GX 클럽 갤럭시S20’ 가입을 고려해 볼 만 하다. 월 이용료 7,480원(부가세 포함)을 내면 24개월 후 새 갤럭시 단말로 기기변경 시 출고가의 50% 수준까지 할인 받는다. SK텔레콤은 3월 멤버십 혜택으로 T Day에 ‘5GX 찬스’를 신설해 5GX 요금제 이용 고객을 대상으로 추가 혜택을 제공한다. 11번가 갤럭시S20 액세서리 22% 할인, 도미노피자 60% 할인 등 갤럭시S20 구매

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [12]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=1024, # electra: 386, bigbird: 1024
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-9583d0011fb0baf6.arrow
Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-569ceaea64024294.arrow


In [14]:
tokenized_squad_augmented = squad_augmented.map(preprocess_function, batched=True, remove_columns=squad_augmented.column_names)

Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-8766c6c9487ce4fd.arrow


In [15]:
data_collator = DefaultDataCollator()
models = []

#model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

urls ='''https://huggingface.co/sunny13/kobigbird-pure32-38554302
https://huggingface.co/alphahg/kobigbird-pure1-10335385
https://huggingface.co/alphahg/kobigbird-pure2-89302097
https://huggingface.co/YuJungSoo/kobigbird-pure25-64664366
https://huggingface.co/sunny13/kobigbird-pure31-53663328'''

urls = urls.split('\n')

for url in urls:
    name = url.split('huggingface.co/')[1]
    models.append(AutoModelForQuestionAnswering.from_pretrained(name))

models

[BigBirdForQuestionAnswering(
   (bert): BigBirdModel(
     (embeddings): BigBirdEmbeddings(
       (word_embeddings): Embedding(32500, 768, padding_idx=0)
       (position_embeddings): Embedding(4096, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BigBirdEncoder(
       (layer): ModuleList(
         (0): BigBirdLayer(
           (attention): BigBirdAttention(
             (self): BigBirdBlockSparseAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
             )
             (output): BigBirdSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [16]:
context = """\
BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
"""
question = "말라카이트에서 나온 색깔을 사용한 에디션은?"
question_answerer = pipeline("question-answering", model=models[0], tokenizer=tokenizer, device=0)
question_answerer(question=question, context=context)

Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


{'score': 0.1705305427312851,
 'start': 666,
 'end': 682,
 'answer': '뉴 740Li 25주년 에디션'}

In [17]:
test_path = './custom_squad_v2/test.json'

test = []
with open(test_path, encoding="utf-8") as f:
    squad_ = json.load(f)
    for example in squad_["data"]:
        title = example.get("title", "")
        for paragraph in example["paragraphs"]:
            context = paragraph["context"]  # do not strip leading blank spaces GH-2585
            for qa in paragraph["qas"]:
                question = qa["question"]
                id_ = qa["guid"]

                # Features currently used are "context", "question", and "answers".
                # Others are extracted here for the ease of future expansions.
                test.append({
                    "title": title,
                    "context": context,
                    "question": question,
                    "id": id_,
                })

In [18]:
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

def get_levenshtein(eval_preds_, answers):
    distances = []
    for i in range(len(eval_preds_)):
        mini = 999999999999
        for ans in answers[i]['text']:
            lev = levenshtein(eval_preds_[i]['answer'], ans)
            
            if mini > lev:
                mini = lev
                
        distances.append(mini)

    mean_distance = sum(distances) / len(distances)
    
    return mean_distance

question_answerers = [pipeline("question-answering", model=model, tokenizer=tokenizer, device=0) for model in models]

In [19]:
def screen_pred(pred):
    length = len(pred['answer'])
    score = pred['score']
    cond = (length > 40 and pred['score'] < 1) or (pred['score'] < 0.5 and length > 20) or pred['score'] < 0.1
    return cond
    # if length > 20 or (pred['score'] < 0.5 and length > 15) or pred['score'] < 0.1:

In [20]:
from copy import deepcopy

In [21]:
preds = []
# preds = [question_answerer(question=data['question'], context=data['context']) for data in tqdm(test)]

for data in tqdm(test):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        preds.append({'answer': '', 'score': 1})
    
    else:
        preds.append({'answer': ans, 'score': score[ans]/maxi})

  0%|          | 0/4008 [00:00<?, ?it/s]Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blo

In [ ]:
preds

[{'score': 0.1348789930343628,
  'start': 666,
  'end': 682,
  'answer': '뉴 740Li 25주년 에디션'},
 {'score': 0.8348477482795715,
  'start': 556,
  'end': 565,
  'answer': '독일 뒤셀도르프로'},
 {'score': 0.39145782589912415, 'start': 436, 'end': 440, 'answer': '트위터가'},
 {'score': 0.49841538071632385, 'start': 537, 'end': 542, 'answer': '링크트인과'},
 {'score': 0.9991483092308044, 'start': 408, 'end': 413, 'answer': '마드리드에'},
 {'score': 0.9427276849746704, 'start': 950, 'end': 956, 'answer': '브라질펀드의'},
 {'score': 0.9822244644165039, 'start': 1352, 'end': 1355, 'answer': '5조원'},
 {'score': 0.9715049266815186,
  'start': 215,
  'end': 225,
  'answer': '‘운영허가 인증서’'},
 {'score': 0.5211303234100342,
  'start': 546,
  'end': 555,
  'answer': '성형 후 가열하여'},
 {'score': 0.9942721128463745, 'start': 320, 'end': 325, 'answer': '노르웨이로'},
 {'score': 0.9591531157493591, 'start': 1146, 'end': 1149, 'answer': '22개'},
 {'score': 0.998506486415863, 'start': 350, 'end': 353, 'answer': '필리핀'},
 {'score': 0.9977768063545227

In [ ]:
test[0]['context'][666:682]

'뉴 740Li 25주년 에디션'

In [ ]:
import pandas as pd

df = pd.read_csv('blank.csv')
blanked = 0
for i in range(len(preds)):
    pred = preds[i]
    length = len(pred['answer'])
    if screen_pred(pred):
        df['Predicted'][i] = ''
        blanked += 1
    else:
        df['Predicted'][i] = pred['answer']

print(f'Blanked preds: {blanked}, total preds: {len(preds)}')

df = df.set_index('Id')

df.to_csv('pred.csv')

In [ ]:

train_preds = []

for data in tqdm(squad['train']):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        train_preds.append({'answer': '', 'score': 1})
    
    else:
        train_preds.append({'answer': ans, 'score': score[ans]/maxi})

In [ ]:
train_preds_ = deepcopy(train_preds)
for i in range(len(train_preds)):
    pred = train_preds_[i]
    if screen_pred(pred):
        train_preds_[i]['answer'] = ''

In [ ]:
train_preds_

In [ ]:
lev_score = get_levenshtein(train_preds_, squad['train']['answers'])
print(f'Train Lev Score: {lev_score}')

In [ ]:
eval = []
eval_preds = []

for data in tqdm(squad['test']):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        eval_preds.append({'answer': '', 'score': 1})
    
    else:
        eval_preds.append({'answer': ans, 'score': score[ans]/maxi})
    

100%|██████████| 1204/1204 [04:41<00:00,  4.28it/s]


In [ ]:
eval_preds

[{'answer': '제2조에는', 'score': 0.4994518533349037},
 {'answer': '가솔린차가', 'score': 0.5701553970575333},
 {'answer': '‘열선 및 통풍시트’', 'score': 0.9533364653587342},
 {'answer': '234건으로', 'score': 0.2787284354368846},
 {'answer': '중종을', 'score': 0.8362335681915283},
 {'answer': '동서횡단철도의', 'score': 0.292882780234019},
 {'answer': '1997년이다', 'score': 0.957687666018804},
 {'answer': '', 'score': 1},
 {'answer': '부천아이파크는', 'score': 0.6696046888828278},
 {'answer': '효원굿플러스를', 'score': 0.7681575417518616},
 {'answer': 'AIA생명보험의', 'score': 0.9306817650794983},
 {'answer': '10년', 'score': 0.9688898126284281},
 {'answer': '', 'score': 1},
 {'answer': '믹수 파텔라이넨 감독은', 'score': 0.5588797852396965},
 {'answer': '‘러비티’', 'score': 0.8766413927078247},
 {'answer': '컨트리', 'score': 0.777279257774353},
 {'answer': '투테크(Tutek)', 'score': 0.6485715011755625},
 {'answer': '', 'score': 1},
 {'answer': '1건에', 'score': 0.936283697684606},
 {'answer': '스쿠트항공이', 'score': 0.6660658195614815},
 {'answer': '', 'score': 1}

In [ ]:
eval_preds_ = deepcopy(eval_preds)

for i in range(len(eval_preds)):
    pred = eval_preds_[i]
    if screen_pred(pred):
        eval_preds_[i]['answer'] = ''

In [ ]:
eval_preds_

[{'answer': '제2조에는', 'score': 0.4994518533349037},
 {'answer': '가솔린차가', 'score': 0.5701553970575333},
 {'answer': '‘열선 및 통풍시트’', 'score': 0.9533364653587342},
 {'answer': '234건으로', 'score': 0.2787284354368846},
 {'answer': '중종을', 'score': 0.8362335681915283},
 {'answer': '동서횡단철도의', 'score': 0.292882780234019},
 {'answer': '1997년이다', 'score': 0.957687666018804},
 {'answer': '', 'score': 1},
 {'answer': '부천아이파크는', 'score': 0.6696046888828278},
 {'answer': '효원굿플러스를', 'score': 0.7681575417518616},
 {'answer': 'AIA생명보험의', 'score': 0.9306817650794983},
 {'answer': '10년', 'score': 0.9688898126284281},
 {'answer': '', 'score': 1},
 {'answer': '믹수 파텔라이넨 감독은', 'score': 0.5588797852396965},
 {'answer': '‘러비티’', 'score': 0.8766413927078247},
 {'answer': '컨트리', 'score': 0.777279257774353},
 {'answer': '투테크(Tutek)', 'score': 0.6485715011755625},
 {'answer': '', 'score': 1},
 {'answer': '1건에', 'score': 0.936283697684606},
 {'answer': '스쿠트항공이', 'score': 0.6660658195614815},
 {'answer': '', 'score': 1}

In [ ]:
lev_score_test = get_levenshtein(eval_preds_, squad['test']['answers'])

print(f'Eval Lev Score: {lev_score_test}')

Eval Lev Score: 2.5921926910299002
